In [6]:
import serial
import time

class AM4100:
    def __init__(self, com_port='COM4', baud_rate=115200):
        self.com_port = com_port
        self.baud_rate = baud_rate
        self.serial_port = None
        self.setup_serial_connection()

    def setup_serial_connection(self):
        try:
            self.serial_port = serial.Serial(self.com_port, self.baud_rate, timeout=10)
            self.serial_port.bytesize = serial.EIGHTBITS
            self.serial_port.stopbits = serial.STOPBITS_ONE
            self.serial_port.parity = serial.PARITY_ODD
            self.serial_port.write_timeout = 10  # Set write timeout to prevent blocking write calls
            time.sleep(1)  # Allow some time for the serial port to initialize
        except serial.SerialException as e:
            print(f"Error opening serial port: {e}")
            self.serial_port = None

    def flush_serial_port(self):
        if self.serial_port is not None:
            self.serial_port.reset_input_buffer()  # Clear input buffer
            self.serial_port.reset_output_buffer()  # Clear output buffer

    def send_command_to_stimulator(self, command):
        if self.serial_port is not None:
            try:
                self.serial_port.write(command.encode() + b'\r\n')
                print(f"Send= {command}")
                time.sleep(0.01)  # Short delay to allow command processing
                response = self.serial_port.readline().decode().strip()
                response = response.replace('\r', '').replace('\n', '~')  # Process response
                print(f"Reply= {response}")
                return response
            except serial.SerialException as e:
                print(f"Error sending command: {e}")
                return "error no data"
        else:
            print("Serial port is not open.")
            return "error no data"
    
    def set_amplitude(self, amplitude):
        command = f"1001 s m 10 7 {amplitude}"
        response = self.send_command_to_stimulator(command)
        return response
    
    def run(self): # this should set it so its waiting for trigger
        command = '1001 s a run'
        response = self.send_command_to_stimulator(command)
    def stop(self): # put it in a ready state but not able to receive trigger
        command = '1001 s a stop'
        response = self.send_command_to_stimulator(command)
        return response

    def close_connection(self):
        if self.serial_port is not open:
            self.serial_port.close()
            print("Serial port closed.")
        else:
            print("Serial port was not open.")

In [2]:
am = AM4100('COM4')
am.flush_serial_port()
#am.close_connection()

In [8]:
am.set_amplitude(amplitude = -25)

Send= 1001 s m 10 7 -25
Reply= 1001 s m 10 7 -25


'1001 s m 10 7 -25'

In [9]:
am.run()

Send= 1001 s a run
Reply= *


In [14]:
am.send_command_to_stimulator('1001 s m 0 0 1')

Send= 1001 s m 0 0 1
Reply= 1001 s m 10 7 -25


'1001 s m 10 7 -25'

In [12]:
am.send_command_to_stimulator('1001 s a stop')

Send= 1001 s a stop
Reply= *


'*'